## Imports

In [ ]:
import os
cwd = os.getcwd()

import sys
sys.path.append(os.path.join(cwd, 'NFF'))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
%matplotlib widget

from ase.io import Trajectory as trajectory
from ase.io import write, read
from ase.optimize import BFGS

In [ ]:
from nff.io.ase import NeuralFF, AtomsBatch
from nff.md.colvars import ColVar
from nff.io.bias_calculators import BiasBase
from nff.train.builders.model import load_model

In [ ]:
import nglview as nv
from copy import deepcopy

In [ ]:
def constrained_optimization(model,
                             atoms, 
                             model_path: str,
                             cv_defs: list[dict],
                             energy_key: str = 'energy_0',
                            ) -> tuple[AtomsBatch, list, list]:

    calculator = BiasBase.from_file(model_path, 
                                cv_defs=cv_defs,
                                directed=True,
                                en_key=energy_key,
                                device='cpu')
    atoms.set_calculator(calculator)
    dyn = BFGS(atoms)

    dyn.run(fmax=0.05, steps=400)
    xi_vals = []
    for cv in cv_defs:
        CV = ColVar(cv['definition'])
        xi , _ = CV(atoms)
        xi_vals.append(xi)
    results = model(atoms.get_batch())
    ener = np.concatenate([results[f'energy_{i}'].detach().numpy() for i in range(3)])
    
    del calculator, dyn
    return atoms, ener, xi_vals

In [ ]:
path_to_model = os.path.join(cwd, 'Pretrained_Models/Model_Adiabatic')

In [ ]:
model = load_model(path_to_model)

In [ ]:
# selecting a starting geometry 
start_config = read('test_frame_0.xyz')

In [ ]:
# needed to tell the ColVar class what CV we want to investigate
# we will use here the dihedral between the two central benzenes
# the CV is in raidans, thus its range is [-pi, pi]
info_dict = {'name': 'distance',
             'index_list': [0, 1],
            }

# value of the starting geometry
CV = ColVar({'name': 'distance',
             'index_list': [0, 1],
            })
xi, xi_grad = CV(start_config) # returns CV values and the gradient w.r.t. to the Cartesian coordinates
print(xi)

CV = ColVar({'name': 'angle',
             'index_list': [0, 1, 3],
            })
xi, xi_grad = CV(start_config) 
print(xi*180/np.pi)

### Example of a constrained minimization

In [ ]:
# to use the ASE-like atoms object as input to a Neral Network
atoms = AtomsBatch.from_atoms(start_config, directed=True, device='cpu')

# this list tells the calculator things about the CV
cv_defs = [{'definition': {'name': 'distance',
             'index_list': [0, 1],
            },
            'range': [1.1, 1.6], 
            'ext_k': 1000.0, # eV / CV^2
            'type': 'not_angle',
            },
           {'definition': {'name': 'angle',
             'index_list': [0, 1, 3],
            },
            'range': [100.0/180.0 * np.pi, 120.0/180.0 * np.pi], 
            'ext_k': 100.0, # eV / CV^2
            'type': 'angle',
            }
            ]

# initialize the calculator from the NFF
# this calculator adds a harmonic spring to the NFF
# the minimum of the sping is at range[0], the force constant is ext_k
calculator = BiasBase.from_file(path_to_model, 
                                cv_defs=cv_defs,
                                directed=True,
                                en_key='energy_0',
                                device='cpu')

atoms.set_calculator(calculator)

dyn = BFGS(atoms)

dyn.run(fmax=0.05, steps=400)

In [ ]:
# proof to what the CV was optimized
# however the bias might have been not strong enough to reach the target ^^
# value of the starting geometry
CV = ColVar({'name': 'distance',
             'index_list': [0, 1],
            })
xi, xi_grad = CV(atoms) # returns CV values and the gradient w.r.t. to the Cartesian coordinates
print(xi)

CV = ColVar({'name': 'angle',
             'index_list': [0, 1, 3],
            })
xi, xi_grad = CV(atoms) 
print(xi*180/np.pi)

In [ ]:
# investigate geometry
nv.show_ase(atoms)

# Comparison of the different model types

### 1D Scan over a range of geometries

In [ ]:
model_names = ['Model_Adiabatic', 'Model_Adiabatic_tuple', 'Model_Diabatic']
results_model_comp = {}
for modelname in model_names:
    path_to_model = os.path.join(cwd, 'Pretrained_Models', modelname)
    model = load_model(path_to_model)
    start_config = read('test_frame_0.xyz')
    atoms = AtomsBatch.from_atoms(start_config, directed=True, device='cpu')

    traj = []
    energies = []
    xis  = []
    
    targets = np.linspace(1.1, 1.6, 41) 
    
    for target in targets:
        print("=======================================================")
        print(f"target: {target}")

        cv_defs = [{'definition': {'name': 'distance',
                                   'index_list': [0, 1],
                                    },
                    'range': [target, target], # second value does not matter here
                    'ext_k': 500.0, # eV / CV^2
                    'type': 'not_angle',
                    }]
        atoms, eners, xi = constrained_optimization(model,
                                                     atoms, 
                                                     model_path = path_to_model,
                                                     cv_defs = cv_defs,
                                                     energy_key = 'energy_0'
                                                   )
        traj.append(deepcopy(atoms))
        xis.append(xi)
        energies.append(eners)
    
    energies = np.array(energies)
    xis = np.array(xis)
    #print(energies)
    results_model_comp[modelname] = {'xis': xis,
                                     'traj': traj,
                                     'energies': energies.reshape(-1, 3)}

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(14,5), 
                        sharex=True, sharey=True)

reds = ['#94003a', '#c82c46', '#ff4e52']
for idx, modelname in enumerate(model_names):
    for idx2 in range(3):  
        axs[idx].plot(results_model_comp[modelname]['xis'], 
                      results_model_comp[modelname]['energies'][:,idx2],
                      label=f'energy_{idx2}',
                      color=reds[idx2],
                      linewidth=2)

for ax in axs:
    ax.tick_params(axis='y',length=6,width=3,labelsize=20, pad=10, direction='in')
    ax.tick_params(axis='x',length=6,width=3,labelsize=20, pad=10, direction='in')
    for key in ax.spines.keys():
        ax.spines[key].set_linewidth(3)
    #ax.set_ylim([0, 0.5])
    ax.set_xlabel("CV value", fontsize=20)
    ax.legend(frameon=False, fontsize=12)

axs[0].set_ylabel(r"Energy / kcal mol$^{-1}$", fontsize=20)

plt.tight_layout()
plt.show()

### 2D Scan over a range of geometries

In [ ]:
model_names = ['Model_Adiabatic', 'Model_Adiabatic_tuple', 'Model_Diabatic']
results_model_comp = {}
for modelname in model_names:
    path_to_model = os.path.join(cwd, 'Pretrained_Models', modelname)
    model = load_model(path_to_model)
    start_config = read('test_frame_0.xyz')
    atoms = AtomsBatch.from_atoms(start_config, directed=True, device='cpu')

    traj = []
    energies = []
    xis  = []
    
    targets_bond = np.linspace(1.1, 1.6, 21) 
    targets_angle = np.linspace(90/180 * np.pi, 150/180 * np.pi, 13) 
    bond_grid, angle_grid = np.meshgrid(targets_bond, targets_angle)
    
    for target_bond, target_angle in zip(bond_grid.flatten(), angle_grid.flatten()):
        print("=======================================================")
        print(f"target bond: {target_bond}, target angel: {target_angle}")

        cv_defs = [{'definition': {'name': 'distance',
                                   'index_list': [0, 1],
                                    },
                    'range': [target_bond, target_bond], # second value does not matter here
                    'ext_k': 500.0, # eV / CV^2
                    'type': 'not_angle',
                    },
                   {'definition': {'name': 'angle',
                                   'index_list': [0, 1, 3],
                                    },
                    'range': [target_angle, target_angle], # second value does not matter here
                    'ext_k': 100.0, # eV / CV^2
                    'type': 'angle',
                    }]
    
        atoms, eners, xi = constrained_optimization(model,
                                                    atoms, 
                                                    model_path = path_to_model,
                                                    cv_defs=cv_defs,
                                                    energy_key = 'energy_0',
                                                   )
        traj.append(deepcopy(atoms))
        xis.append(xi)
        energies.append(eners)
    
    energies = np.array(energies)
    xis = np.array(xis)
    results_model_comp[modelname] = {'xis': xis,
                                     'traj': traj,
                                     'energies': energies.reshape(-1, 3)}

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
X = results_model_comp['Model_Adiabatic']['xis'][:,0]#.reshape(targets_bond.shape[0], targets_angle.shape[0])
Y = results_model_comp['Model_Adiabatic']['xis'][:,1]#.reshape(targets_bond.shape[0], targets_angle.shape[0])
Z = results_model_comp['Model_Adiabatic']['energies'][:,1]#.reshape(targets_bond.shape[0], targets_angle.shape[0])
surf = ax.plot_trisurf(X, Y, Z, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

Z = results_model_comp['Model_Adiabatic']['energies'][:,2]#.reshape(targets_bond.shape[0], targets_angle.shape[0])
surf = ax.plot_trisurf(X, Y, Z, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

ax.set_xlabel("C-N bond length / A")
ax.set_ylabel("C-N-H angle / rad")
ax.set_zlabel("Energy / kcal mol$^{-1}$")
plt.show()

In [ ]:
model_type = 'Model_Diabatic'

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
X = results_model_comp[model_type]['xis'][:,0]#.reshape(targets_bond.shape[0], targets_angle.shape[0])
Y = results_model_comp[model_type]['xis'][:,1]#.reshape(targets_bond.shape[0], targets_angle.shape[0])
Z = results_model_comp[model_type]['energies'][:,1]#.reshape(targets_bond.shape[0], targets_angle.shape[0])
surf = ax.plot_trisurf(X, Y, Z, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

Z = results_model_comp[model_type]['energies'][:,2]#.reshape(targets_bond.shape[0], targets_angle.shape[0])
surf = ax.plot_trisurf(X, Y, Z, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

ax.set_xlabel("C-N bond length / A")
ax.set_ylabel("C-N-H angle / rad")
ax.set_zlabel("Energy / kcal mol$^{-1}$")
plt.show()